# Crawling은 Local에서 수행함

In [ ]:
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup
import requests

import os
import warnings
warnings.filterwarnings("ignore")


import time
#time.sleep을 위한 난수생성
import random

In [ ]:
df = pd.read_excel("만개의 레시피1.xlsx")
df.head()

,RCP_SNO,RCP_TTL,CKG_NM,RGTR_ID,RGTR_NM,INQ_CNT,RCMM_CNT,SRAP_CNT,CKG_MTH_ACTO_NM,CKG_STA_ACTO_NM,CKG_MTRL_ACTO_NM,CKG_KND_ACTO_NM,CKG_IPDC,CKG_MTRL_CN,CKG_INBUN_NM,CKG_DODF_NM,CKG_TIME_NM,FIRST_REG_DT
0,128671,어묵김말이,어묵김말이,skfo0701,꽃날,9592,6,66,튀김,간식,가공식품류,디저트,맛있는 김말이에 쫄깃함을 더한 어묵 김말이예요-,[재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개|...,2인분,초급,60분이내,20070402131403
1,128892,두부에 꼬리가 달렸어요!!,NaN,skfo0701,꽃날,5538,3,26,부침,일상,해물류,밑반찬,꼬리가 너-무- 매력적인 두부새우전. 두부와 야채를 한번에!! 영양까지 만점인 두부...,[재료] 두부 1/2모| 당근 1/2개| 고추 2개| 브로콜리 1/4개| 새우 4마...,3인분,초급,30분이내,20070402205937
2,128932,입안에서 톡톡톡,NaN,skfo0701,꽃날,6802,8,36,굽기,일상,해물류,밥/죽/떡,간단하게 만들어 보는 알이 톡톡톡 알밥♥ 다 먹고 누룽지까지 싹싹 긁어먹는게 최고죠...,[재료] 밥 1+1/2공기| 당근 1/4개| 치자단무지 1/2개| 신김치 1쪽| 무...,2인분,초급,30분이내,20070402224355
3,131871,★현미호두죽,현미호두죽,cds1117,햇님&별님,2912,0,9,끓이기,일상,쌀,밥/죽/떡,현미호두죽,[재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간,2인분,초급,30분이내,20070410142301
4,139247,부들부들 보들보들 북어갈비♥,북어갈비,skfo0701,꽃날,6865,3,97,굽기,술안주,건어물류,메인반찬,오늘은 집에서 굴러다니고 쉽게 구할 수 있는 북어로 일품요리를 만들어 보았어요! 도...,[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T|...,2인분,초급,60분이내,20070501000844


In [ ]:
len(df)

128400

In [ ]:
df.dropna(inplace=True)
df.reset_index(drop = True, inplace=True)
print(len(df))

114225


In [ ]:
df.head()

,RCP_SNO,RCP_TTL,CKG_NM,RGTR_ID,RGTR_NM,INQ_CNT,RCMM_CNT,SRAP_CNT,CKG_MTH_ACTO_NM,CKG_STA_ACTO_NM,CKG_MTRL_ACTO_NM,CKG_KND_ACTO_NM,CKG_IPDC,CKG_MTRL_CN,CKG_INBUN_NM,CKG_DODF_NM,CKG_TIME_NM,FIRST_REG_DT
0,128671,어묵김말이,어묵김말이,skfo0701,꽃날,9592,6,66,튀김,간식,가공식품류,디저트,맛있는 김말이에 쫄깃함을 더한 어묵 김말이예요-,[재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개|...,2인분,초급,60분이내,20070402131403
1,131871,★현미호두죽,현미호두죽,cds1117,햇님&별님,2912,0,9,끓이기,일상,쌀,밥/죽/떡,현미호두죽,[재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간,2인분,초급,30분이내,20070410142301
2,139247,부들부들 보들보들 북어갈비♥,북어갈비,skfo0701,꽃날,6865,3,97,굽기,술안주,건어물류,메인반찬,오늘은 집에서 굴러다니고 쉽게 구할 수 있는 북어로 일품요리를 만들어 보았어요! 도...,[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T|...,2인분,초급,60분이내,20070501000844
3,149207,토마토스파게티♥,토마토스파게티,skfo0701,꽃날,12754,2,36,볶음,일상,가공식품류,면/만두,한번 만들어 두면 이모저모 쓸 용도가 많은 토마토소스를 이용한 토마토 스파게티.,[재료] 파스타면 [양념] 토마토 1개| 토마토 페이스트 3T| 양파 1/2개| 다...,1인분,초급,30분이내,20070525232601
4,151148,표고버섯탕수,표고버섯탕수,skfo0701,꽃날,16053,2,194,튀김,손님접대,버섯류,메인반찬,탕수육 먹을때 칼로리때문에 걱정하시는 분들!! 이제는 걱걱걱 정하지 마세요~ 칼로리...,[재료] 건표고버섯 9개| 오이 1/2개| 당근 1/2개| 양파 1/2개| 사과 1...,2인분,초급,30분이내,20070530142653


In [ ]:
datadir = 'D:/song/project/project3/data/traindata/한국 음식 이미지'

food_middle_list = os.listdir(datadir)
food_middle_list = sorted(food_middle_list)
food_middle_list[:5]

['구이', '국', '기타', '김치', '나물']

In [ ]:
food_list = [] # 음식이름
food_path_list = [] # 음식이름 경로

for food_middle in food_middle_list:
    middle_path = os.path.join(datadir,food_middle)
    food_name_list = os.listdir(middle_path)
    for food in food_name_list:
        path = os.path.join(middle_path,food)
        food_path_list.append(path)

        name = path.split('\\')[-1]
        food_list.append(name)

print(len(food_list))
print(food_list)
print(food_path_list)

150
['갈비구이', '갈치구이', '고등어구이', '곱창구이', '닭갈비', '더덕구이', '떡갈비', '불고기', '삼겹살', '장어구이', '조개구이', '조기구이', '황태구이', '훈제오리', '계란국', '떡국_만두국', '무국', '미역국', '북엇국', '시래기국', '육개장', '콩나물국', '과메기', '양념치킨', '젓갈', '콩자반', '편육', '피자', '후라이드치킨', '갓김치', '깍두기', '나박김치', '무생채', '배추김치', '백김치', '부추김치', '열무김치', '오이소박이', '총각김치', '파김치', '가지볶음', '고사리나물', '미역줄기볶음', '숙주나물', '시금치나물', '애호박볶음', '경단', '꿀떡', '송편', '만두', '라면', '막국수', '물냉면', '비빔냉면', '수제비', '열무국수', '잔치국수', '짜장면', '짬뽕', '쫄면', '칼국수', '콩국수', '꽈리고추무침', '도라지무침', '도토리묵', '잡채', '콩나물무침', '홍어무침', '회무침', '김밥', '김치볶음밥', '누룽지', '비빔밥', '새우볶음밥', '알밥', '유부초밥', '잡곡밥', '주먹밥', '감자채볶음', '건새우볶음', '고추장진미채볶음', '두부김치', '떡볶이', '라볶이', '멸치볶음', '소세지볶음', '어묵볶음', '오징어채볶음', '제육볶음', '주꾸미볶음', '보쌈', '수정과', '식혜', '간장게장', '양념게장', '깻잎장아찌', '떡꼬치', '감자전', '계란말이', '계란후라이', '김치전', '동그랑땡', '생선전', '파전', '호박전', '곱창전골', '갈치조림', '감자조림', '고등어조림', '꽁치조림', '두부조림', '땅콩조림', '메추리알장조림', '연근조림', '우엉조림', '장조림', '코다리조림', '전복죽', '호박죽', '김치찌개', '닭계장', '동태찌개', '된장찌개', '순두부찌개', '갈비찜', '계란찜', '김치찜', '꼬막찜', '닭볶음탕', '수육'

In [ ]:
## 정확히 이름이 일치할때

temp = df.copy()
temp = temp[['RCP_SNO', 'CKG_NM']]

for idx, food in enumerate(food_list):
    temp[f'food{idx}'] = temp['CKG_NM'].apply(lambda x: 1 if (x == food) else 0)

temp['select'] = temp.iloc[:, 2: ].sum(axis = 1)


### 포함된 이름을 하고 싶을때

# temp = df.copy()
# temp = temp[['RCP_SNO', 'CKG_NM']]

# for idx, food in enumerate(food_list):
#     temp[f'food{idx}'] = temp['CKG_NM'].apply(lambda x: 1 if (x.find(food) > -1) else 0)

# temp['select'] = temp.iloc[:, 2: ].sum(axis = 1)

# temp = temp[temp['select'] == 1]
# df_select = temp[['RCP_SNO', 'CKG_NM', 'select']]
# df_select.head()

# df_select.shape

In [ ]:
temp = temp[temp['select'] == 1]
df_select = temp[['RCP_SNO', 'CKG_NM', 'select']]
df_select.head()

,RCP_SNO,CKG_NM,select
29,349381,두부조림,1
31,349740,어묵볶음,1
42,380726,찜닭,1
85,414544,김치볶음밥,1
93,421942,어묵볶음,1


In [ ]:
print(df_select.shape)

(11176, 3)


In [ ]:
df_merge = df_select.merge(df,
                           how ='left',
                           on = ['RCP_SNO', 'CKG_NM'])

In [ ]:
df_merge.shape

(11176, 19)

In [ ]:
df_merge = df_merge.sort_values(by='RCMM_CNT', ascending=False)
df_merge.reset_index(drop = True, inplace=True)
df_merge.head()

,RCP_SNO,CKG_NM,select,RCP_TTL,RGTR_ID,RGTR_NM,INQ_CNT,RCMM_CNT,SRAP_CNT,CKG_MTH_ACTO_NM,CKG_STA_ACTO_NM,CKG_MTRL_ACTO_NM,CKG_KND_ACTO_NM,CKG_IPDC,CKG_MTRL_CN,CKG_INBUN_NM,CKG_DODF_NM,CKG_TIME_NM,FIRST_REG_DT
0,6852249,김치볶음밥,1,백전백승! 실패없는 김치볶음밥,gdubu33,만개의레시피,366065,1324,6914,볶음,일상,쌀,밥/죽/떡,이 레시피대로만 따라하면 절대! 실패따윈 두려워 하지 않아도 되! 세상의 모든 레...,[필수재료] 밥 1공기| 다진 양파 3큰술| 다진 김치 1/2컵| 다진 햄 3큰술 ...,1인분,아무나,15분이내,20160706123205
1,6859485,감자조림,1,매일 먹는 밑반찬 메뉴로 추천!! 감자조림,gdubu33,만개의레시피,478527,1102,8221,조림,일상,채소류,밑반찬,만들어두면 마음&배가 든든해지는 달큰 짭잘 감자조림! 세상의 모든 레시피 만개의레시피,[필수재료] 감자 2개(360g)| 물 1cup [양념재료] 간장 5T| 설탕 1T...,2인분,아무나,30분이내,20161031112035
2,6846655,김치찌개,1,새마을식당 7분김치찌개 만들기,76918586,임루시,218649,1077,7594,끓이기,일상,채소류,찌개,새마을식당 김치찌개 따라잡기,[재료] 김치| 돼지고기(찌개용)| 양파 1/2개| 파 약간 [양념] 김치국물 2~...,2인분,아무나,15분이내,20160418201815
3,6841008,제육볶음,1,백종원 제육볶음 레시피^^ 백종원의 노하우가 들어있는 손쉬운 레시피 백종원 제육볶음...,only1a2a3a,랑이의보물상자,976200,915,12418,볶음,술안주,돼지고기,메인반찬,백종원의 제육볶음 레시피로 간단하게 밥반찬 또는 야식 술안주 뚝딱 만들어봤습니다^^,[재료] 돼지고기 600g| 양파 1개| 청양고추 2개| 대파 1뿌리 [양념] 설탕...,4인분,아무나,30분이내,20160103095322
4,6857726,고등어조림,1,비린내 걱정없는 백종원 고등어조림,heeya120608,심술이네집밥이야기,297242,887,10924,조림,일상,해물류,메인반찬,집밥 백선생 시즌 2에서 소개된 비린내 없이 맛있게 만들수 있는 백종원씨의 고등어...,[재료] 고등어 2마리| 무| 양파| 대파| 청양고추 [양념] 설탕 1큰술| 다진마...,4인분,초급,60분이내,20160930162922


In [ ]:
len(df_merge)

11176

In [ ]:
cnt = 0
step1 = 500
for i in range(0, len(df_merge), step1):
    cnt += 1
    if i != 11000:
        globals()[f'part{cnt}'] = df_merge[i:i+step1]
    else :
        globals()[f'part{cnt}'] = df_merge[i:]

In [ ]:
for data_num in range(1, 4):
    results = []
    temp = globals()[f'part{data_num}']

    recipe = []

    base_url = 'https://www.10000recipe.com/recipe/'
    for idx in temp.index:
        recipe_num = temp['RCP_SNO'][idx]
        url = base_url + str(recipe_num)

        time.sleep(round(random.uniform(0.4, 1.2), 3))

        html = requests.get(url).text
        soup = BeautifulSoup(html, "html.parser")

        if random.randint(1, 3) == 2:
            time.sleep(round(random.uniform(0.4, 1.2), 3))

        step = []
        step_raw = soup.find_all("div", {"class": "view_step_cont"})
        for i in range(len(step_raw)):
            step.append(f'step:' + step_raw[i].text.strip())

        recipe.append({'RCP_SNO' : recipe_num,
                       'step' : step})

        time.sleep(round(random.uniform(0.4, 1.2), 3))

    df_recipe = pd.DataFrame(recipe)
    df_recipe.to_csv(f'steps/step{data_num}.csv', index=False)
    df_recipe.to_excel(f'steps/step{data_num}.xlsx', index = False)

    time.sleep(round(random.uniform(30, 65), 3))

# recipe = []

# base_url = 'https://www.10000recipe.com/recipe/'
# for idx in range(len(temp)):
#     recipe_num = temp['RCP_SNO'][idx]
#     url = base_url + str(recipe_num)

#     time.sleep(round(random.uniform(0.4, 1.2), 3))

#     html = requests.get(url).text
#     soup = BeautifulSoup(html, "html.parser")

#     step = []
#     step_raw = soup.find_all("div", {"class": "view_step_cont"})
#     for i in range(len(step_raw)):
#         step.append(f'step:' + step_raw[i].text.strip())

#     recipe.append({'RCP_SNO' : recipe_num,
#                    'step' : step})

#     time.sleep(round(random.uniform(0.4, 1.2), 3))

# df_recipe = pd.DataFrame(recipe)
# df_recipe.to_csv('step.csv', index=False)
# df_recipe.to_excel('step.xlsx', index = False)

In [ ]:
datadir = 'steps/'

file_list = os.listdir(datadir)
file_list = sorted(file_list)
selected_csv =[csv for csv in file_list if csv.split('.')[-1] == 'csv']
selected_csv

['step1.csv', 'step2.csv', 'step3.csv']

In [ ]:
df_recipe.columns

Index(['RCP_SNO', 'step'], dtype='object')

In [ ]:
df_step = pd.DataFrame(columns = ['RCP_SNO', 'step'])

for file_name in selected_csv:
    folder_path = datadir
    path = folder_path + file_name

    temp = pd.read_csv(path)

    df_step = pd.concat([df_step, temp], axis = 0)

In [ ]:
df_step.head()

,RCP_SNO,step
0,6852249,"['step:밥은 넓은 접시에 펼쳐 뜨거운 김을 날린다.밥 1공기', 'step:양..."
1,6859485,['step:감자는 손질하여 한입 크기로 썰어 준비한다.감자 2개(360g)감자 손...
2,6846655,['step:물 한 컵에 돼지고기를 넣고 끓여주세요\n물 대신 쌀뜨물을 사용하시면 ...
3,6841008,"['step:재료는 먹기 좋은 크기도 썰어서 준비해주세요^^', 'step:고추장 ..."
4,6857726,['step:고등어는 내장을 제거하시고 \n핏물이 남지 않도록 깨끗이 씻어 준비합니...


In [ ]:
df = pd.read_excel("만개의 레시피1.xlsx")
df.head()

,RCP_SNO,RCP_TTL,CKG_NM,RGTR_ID,RGTR_NM,INQ_CNT,RCMM_CNT,SRAP_CNT,CKG_MTH_ACTO_NM,CKG_STA_ACTO_NM,CKG_MTRL_ACTO_NM,CKG_KND_ACTO_NM,CKG_IPDC,CKG_MTRL_CN,CKG_INBUN_NM,CKG_DODF_NM,CKG_TIME_NM,FIRST_REG_DT
0,128671,어묵김말이,어묵김말이,skfo0701,꽃날,9592,6,66,튀김,간식,가공식품류,디저트,맛있는 김말이에 쫄깃함을 더한 어묵 김말이예요-,[재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개|...,2인분,초급,60분이내,20070402131403
1,128892,두부에 꼬리가 달렸어요!!,NaN,skfo0701,꽃날,5538,3,26,부침,일상,해물류,밑반찬,꼬리가 너-무- 매력적인 두부새우전. 두부와 야채를 한번에!! 영양까지 만점인 두부...,[재료] 두부 1/2모| 당근 1/2개| 고추 2개| 브로콜리 1/4개| 새우 4마...,3인분,초급,30분이내,20070402205937
2,128932,입안에서 톡톡톡,NaN,skfo0701,꽃날,6802,8,36,굽기,일상,해물류,밥/죽/떡,간단하게 만들어 보는 알이 톡톡톡 알밥♥ 다 먹고 누룽지까지 싹싹 긁어먹는게 최고죠...,[재료] 밥 1+1/2공기| 당근 1/4개| 치자단무지 1/2개| 신김치 1쪽| 무...,2인분,초급,30분이내,20070402224355
3,131871,★현미호두죽,현미호두죽,cds1117,햇님&별님,2912,0,9,끓이기,일상,쌀,밥/죽/떡,현미호두죽,[재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간,2인분,초급,30분이내,20070410142301
4,139247,부들부들 보들보들 북어갈비♥,북어갈비,skfo0701,꽃날,6865,3,97,굽기,술안주,건어물류,메인반찬,오늘은 집에서 굴러다니고 쉽게 구할 수 있는 북어로 일품요리를 만들어 보았어요! 도...,[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T|...,2인분,초급,60분이내,20070501000844


In [ ]:
df_final = df_step.merge(df,
                         how = 'left',
                         on = 'RCP_SNO')

df_final.head()

,RCP_SNO,step,RCP_TTL,CKG_NM,RGTR_ID,RGTR_NM,INQ_CNT,RCMM_CNT,SRAP_CNT,CKG_MTH_ACTO_NM,CKG_STA_ACTO_NM,CKG_MTRL_ACTO_NM,CKG_KND_ACTO_NM,CKG_IPDC,CKG_MTRL_CN,CKG_INBUN_NM,CKG_DODF_NM,CKG_TIME_NM,FIRST_REG_DT
0,6852249,"['step:밥은 넓은 접시에 펼쳐 뜨거운 김을 날린다.밥 1공기', 'step:양...",백전백승! 실패없는 김치볶음밥,김치볶음밥,gdubu33,만개의레시피,366065,1324,6914,볶음,일상,쌀,밥/죽/떡,이 레시피대로만 따라하면 절대! 실패따윈 두려워 하지 않아도 되! 세상의 모든 레...,[필수재료] 밥 1공기| 다진 양파 3큰술| 다진 김치 1/2컵| 다진 햄 3큰술 ...,1인분,아무나,15분이내,20160706123205
1,6859485,['step:감자는 손질하여 한입 크기로 썰어 준비한다.감자 2개(360g)감자 손...,매일 먹는 밑반찬 메뉴로 추천!! 감자조림,감자조림,gdubu33,만개의레시피,478527,1102,8221,조림,일상,채소류,밑반찬,만들어두면 마음&배가 든든해지는 달큰 짭잘 감자조림! 세상의 모든 레시피 만개의레시피,[필수재료] 감자 2개(360g)| 물 1cup [양념재료] 간장 5T| 설탕 1T...,2인분,아무나,30분이내,20161031112035
2,6846655,['step:물 한 컵에 돼지고기를 넣고 끓여주세요\n물 대신 쌀뜨물을 사용하시면 ...,새마을식당 7분김치찌개 만들기,김치찌개,76918586,임루시,218649,1077,7594,끓이기,일상,채소류,찌개,새마을식당 김치찌개 따라잡기,[재료] 김치| 돼지고기(찌개용)| 양파 1/2개| 파 약간 [양념] 김치국물 2~...,2인분,아무나,15분이내,20160418201815
3,6841008,"['step:재료는 먹기 좋은 크기도 썰어서 준비해주세요^^', 'step:고추장 ...",백종원 제육볶음 레시피^^ 백종원의 노하우가 들어있는 손쉬운 레시피 백종원 제육볶음...,제육볶음,only1a2a3a,랑이의보물상자,976200,915,12418,볶음,술안주,돼지고기,메인반찬,백종원의 제육볶음 레시피로 간단하게 밥반찬 또는 야식 술안주 뚝딱 만들어봤습니다^^,[재료] 돼지고기 600g| 양파 1개| 청양고추 2개| 대파 1뿌리 [양념] 설탕...,4인분,아무나,30분이내,20160103095322
4,6857726,['step:고등어는 내장을 제거하시고 \n핏물이 남지 않도록 깨끗이 씻어 준비합니...,비린내 걱정없는 백종원 고등어조림,고등어조림,heeya120608,심술이네집밥이야기,297242,887,10924,조림,일상,해물류,메인반찬,집밥 백선생 시즌 2에서 소개된 비린내 없이 맛있게 만들수 있는 백종원씨의 고등어...,[재료] 고등어 2마리| 무| 양파| 대파| 청양고추 [양념] 설탕 1큰술| 다진마...,4인분,초급,60분이내,20160930162922


In [ ]:
print(len(df_final))

1500


In [ ]:
metadata = pd.read_excel('만개의 레시피_데이터명세서.xlsx')
metadata.head()

In [ ]:
recipe_dict = dict(zip(metadata['nameEn'], metadata['nameKo']))
recipe_dict

{'RCP_SNO': '레시피일련번호',
 'RCP_TTL': '레시피제목',
 'CKG_NM': '요리명',
 'RGTR_ID': '등록자ID',
 'RGTR_NM': '등록자명',
 'INQ_CNT': '조회수',
 'RCMM_CNT': '추천수',
 'SRAP_CNT': '스크랩수',
 'CKG_MTH_ACTO_NM': '요리방법별명',
 'CKG_STA_ACTO_NM': '요리상황별명',
 'CKG_MTRL_ACTO_NM': '요리재료별명',
 'CKG_KND_ACTO_NM': '요리종류별명',
 'CKG_IPDC': '요리소개',
 'CKG_MTRL_CN': '요리재료내용',
 'CKG_INBUN_NM': '요리인분명',
 'CKG_DODF_NM': '요리난이도명',
 'CKG_TIME_NM': '요리시간명',
 'FIRST_REG_DT': '최초등록일시'}

In [ ]:
df_final = df_step.merge(df,
                         how = 'left',
                         on = 'RCP_SNO')


df_final.rename(columns=recipe_dict, inplace=True)
df_final.dropna(inplace=True)

df_final.head()

,레시피일련번호,step,레시피제목,요리명,등록자ID,등록자명,조회수,추천수,스크랩수,요리방법별명,요리상황별명,요리재료별명,요리종류별명,요리소개,요리재료내용,요리인분명,요리난이도명,요리시간명,최초등록일시
0,6852249,"['step:밥은 넓은 접시에 펼쳐 뜨거운 김을 날린다.밥 1공기', 'step:양...",백전백승! 실패없는 김치볶음밥,김치볶음밥,gdubu33,만개의레시피,366065,1324,6914,볶음,일상,쌀,밥/죽/떡,이 레시피대로만 따라하면 절대! 실패따윈 두려워 하지 않아도 되! 세상의 모든 레...,[필수재료] 밥 1공기| 다진 양파 3큰술| 다진 김치 1/2컵| 다진 햄 3큰술 ...,1인분,아무나,15분이내,20160706123205
1,6859485,['step:감자는 손질하여 한입 크기로 썰어 준비한다.감자 2개(360g)감자 손...,매일 먹는 밑반찬 메뉴로 추천!! 감자조림,감자조림,gdubu33,만개의레시피,478527,1102,8221,조림,일상,채소류,밑반찬,만들어두면 마음&배가 든든해지는 달큰 짭잘 감자조림! 세상의 모든 레시피 만개의레시피,[필수재료] 감자 2개(360g)| 물 1cup [양념재료] 간장 5T| 설탕 1T...,2인분,아무나,30분이내,20161031112035
2,6846655,['step:물 한 컵에 돼지고기를 넣고 끓여주세요\n물 대신 쌀뜨물을 사용하시면 ...,새마을식당 7분김치찌개 만들기,김치찌개,76918586,임루시,218649,1077,7594,끓이기,일상,채소류,찌개,새마을식당 김치찌개 따라잡기,[재료] 김치| 돼지고기(찌개용)| 양파 1/2개| 파 약간 [양념] 김치국물 2~...,2인분,아무나,15분이내,20160418201815
3,6841008,"['step:재료는 먹기 좋은 크기도 썰어서 준비해주세요^^', 'step:고추장 ...",백종원 제육볶음 레시피^^ 백종원의 노하우가 들어있는 손쉬운 레시피 백종원 제육볶음...,제육볶음,only1a2a3a,랑이의보물상자,976200,915,12418,볶음,술안주,돼지고기,메인반찬,백종원의 제육볶음 레시피로 간단하게 밥반찬 또는 야식 술안주 뚝딱 만들어봤습니다^^,[재료] 돼지고기 600g| 양파 1개| 청양고추 2개| 대파 1뿌리 [양념] 설탕...,4인분,아무나,30분이내,20160103095322
4,6857726,['step:고등어는 내장을 제거하시고 \n핏물이 남지 않도록 깨끗이 씻어 준비합니...,비린내 걱정없는 백종원 고등어조림,고등어조림,heeya120608,심술이네집밥이야기,297242,887,10924,조림,일상,해물류,메인반찬,집밥 백선생 시즌 2에서 소개된 비린내 없이 맛있게 만들수 있는 백종원씨의 고등어...,[재료] 고등어 2마리| 무| 양파| 대파| 청양고추 [양념] 설탕 1큰술| 다진마...,4인분,초급,60분이내,20160930162922


In [ ]:
print(df_final.shape)

(1500, 19)


In [ ]:
import re
df_final['요리재료내용'].apply(lambda x: x.split('['))



0       [, 필수재료] 밥 1공기| 다진 양파 3큰술| 다진 김치 1/2컵| 다진 햄 3큰...
1       [, 필수재료] 감자 2개(360g)| 물 1cup , 양념재료] 간장 5T| 설탕...
2       [, 재료] 김치| 돼지고기(찌개용)| 양파 1/2개| 파 약간 , 양념] 김치국물...
3       [, 재료] 돼지고기 600g| 양파 1개| 청양고추 2개| 대파 1뿌리 , 양념]...
4       [, 재료] 고등어 2마리| 무| 양파| 대파| 청양고추 , 양념] 설탕 1큰술| ...
                              ...                        
1495    [, 재료] 김치 1/4쪽| 밥 2공기| 모짜렐라치즈 1/2컵| 계란 2개| 들기름...
1496    [, 재료] 라면 1개| 물 500cc| 대파 1줄 , 양념] 다진생강 1작은술| ...
1497    [, 재료] 닭| 김칫국물| 고구마| 당근| 양파| 파 , 양념] 고추장 4큰술| ...
1498    [, 재료] 우엉 400g| 조청 100ml(1/2컵)| 맛간장/양조간장 3큰술| ...
1499    [, 재료] 불린당면 300g| 돌려깍은오이 1개| 당근 1/2개| 목이버섯| 새송...
Name: 요리재료내용, Length: 1500, dtype: object

In [ ]:
split_data = df_final['요리재료내용'].apply(lambda x: x.split('['))
new_df = pd.DataFrame(split_data.tolist(), columns=[f'Column_{i}' for i in range(8)])


In [ ]:
new_df

,Column_0,Column_1,Column_2,Column_3,Column_4,Column_5,Column_6,Column_7
0,,필수재료] 밥 1공기| 다진 양파 3큰술| 다진 김치 1/2컵| 다진 햄 3큰술,양념장] 설탕 2/3큰술| 김칫국물 2큰술| 고추장 2/3큰술| 부순 참깨 1큰술|...,None,None,None,None,None
1,,필수재료] 감자 2개(360g)| 물 1cup,양념재료] 간장 5T| 설탕 1T| 물엿 2T| 다진 마늘 1/2T| 참깨 1/2T...,None,None,None,None,None
2,,재료] 김치| 돼지고기(찌개용)| 양파 1/2개| 파 약간,양념] 김치국물 2~3큰술| 간장 1큰술| 고춧가루 1큰술| 다진마늘 1/2큰술| ...,None,None,None,None,None
3,,재료] 돼지고기 600g| 양파 1개| 청양고추 2개| 대파 1뿌리,양념] 설탕 2스푼| 고추장 2스푼| 간장 2스푼| 고춧가루 2스푼| 다진마늘 1스...,None,None,None,None,None
4,,재료] 고등어 2마리| 무| 양파| 대파| 청양고추,양념] 설탕 1큰술| 다진마늘 1+1/2큰술| 다진생강 1/3큰술| 된장 1/2~1...,None,None,None,None,None
...,...,...,...,...,...,...,...,...
1495,,재료] 김치 1/4쪽| 밥 2공기| 모짜렐라치즈 1/2컵| 계란 2개| 들기름 2큰술,양념] 고추가루 1큰술,None,None,None,None,None
1496,,재료] 라면 1개| 물 500cc| 대파 1줄,양념] 다진생강 1작은술| 다진마늘 1작은술| 고춧가루 1숟갈| 참기름 1/2숟갈|...,None,None,None,None,None
1497,,재료] 닭| 김칫국물| 고구마| 당근| 양파| 파,양념] 고추장 4큰술| 고춧가루 2큰술| 올리고당 3큰술| 설탕| 간장 8큰술정도|...,None,None,None,None,None
1498,,재료] 우엉 400g| 조청 100ml(1/2컵)| 맛간장/양조간장 3큰술| 국간장...,None,None,None,None,None,None


In [ ]:
test =pd.DataFrame()




new_df['Column_0'].apply(lambda x: x.split(r'\]')[0])


0        
1        
2        
3        
4        
       ..
1495     
1496     
1497     
1498     
1499     
Name: Column_0, Length: 1500, dtype: object

In [ ]:
new_df['Column_1'].apply(lambda x: x.split(r'\]')[1])

IndexError: list index out of range

In [ ]:
df_final.to_csv('recipe_data.csv')
df_final.to_excel('recipe_data.xlsx')